In [19]:
import csv
import json
import os
import tempfile
import subprocess
import sys
from typing import Dict, List, Tuple, Any
import time
from datetime import datetime









###################### Configuration variables (customize these) ###################### 
### Choose one - CSV or JSON
### Comment the other


CSV_FILE_NAME = 'mHumanEval-mini.csv'  # The CSV file to evaluate
#JSON_FILE_NAME = 'mHumanEval-ben_Beng.json'  # The JSON file to evaluate
SOLUTION_COLUMN = 'canonical_solution'  # Column containing the solutions to evaluate


###################### ###################### ###################### ##################








def extract_docstring(prompt: str) -> str:
    """Extract the docstring from a function prompt."""
    try:
        docstring_start = prompt.index('"""')
        docstring_end = prompt.index('"""', docstring_start + 3)
        return prompt[docstring_start:docstring_end + 3]
    except ValueError:
        # Try single quotes if double quotes aren't found
        try:
            docstring_start = prompt.index("'''")
            docstring_end = prompt.index("'''", docstring_start + 3)
            return prompt[docstring_start:docstring_end + 3]
        except ValueError:
            return ""

def create_test_file(task_id: str, prompt: str, solution: str, test: str) -> str:
    """Create a temporary Python file containing the solution and tests."""
    with tempfile.NamedTemporaryFile(suffix='.py', delete=False, mode='w', encoding='utf-8') as f:
        # Extract the docstring to preserve it
        docstring = extract_docstring(prompt)
        
        # Get the function signature (everything before the docstring)
        function_signature = prompt[:prompt.index(docstring) if docstring else len(prompt)]
        
        # Write the function definition with docstring
        f.write(function_signature)
        f.write(docstring)
        f.write("\n")
        
        # Write the solution (indented based on the function signature)
        if not solution.startswith(" ") and not solution.startswith("\t") and function_signature.strip().endswith(":"):
            # Add indentation if the solution doesn't have it
            solution_lines = solution.strip().split("\n")
            indented_solution = "\n".join(f"    {line}" for line in solution_lines)
            f.write(indented_solution)
        else:
            f.write(solution)
        
        # Add a newline before tests
        f.write("\n\n")
        
        # Add test code
        f.write(test)
        
        # Add execution code
        f.write("\n\n# Execute tests\n")
        f.write("if __name__ == '__main__':\n")
        f.write("    try:\n")
        f.write("        check(")
        
        # Extract function name from prompt
        function_name = None
        for line in function_signature.split("\n"):
            if line.strip().startswith("def "):
                function_name = line.strip().split("def ")[1].split("(")[0].strip()
                break
        
        if function_name:
            f.write(function_name)
            f.write(")\n")
            f.write("        print('PASS')\n")
            f.write("    except AssertionError as e:\n")
            f.write("        print(f'FAIL: {e}')\n")
            f.write("    except Exception as e:\n")
            f.write("        print(f'ERROR: {e}')\n")
        else:
            f.write("# Could not find function name in prompt\n")
            f.write("        print('FAIL: Could not find function name')\n")
    
    return f.name

def run_test(test_file_path: str) -> Tuple[str, bool]:
    """Run the test file and return the result and whether it passed."""
    try:
        result = subprocess.run(
            [sys.executable, test_file_path],
            capture_output=True,
            text=True,
            timeout=5  # 5 second timeout
        )
        output = result.stdout.strip() + result.stderr.strip()
        passed = "PASS" in output and "FAIL" not in output and "ERROR" not in output
        return output, passed
    except subprocess.TimeoutExpired:
        return "TIMEOUT", False
    except Exception as e:
        return f"EXECUTION_ERROR: {e}", False

def find_data_file():
    """Find the data file by checking for valid files."""
    # List of possible file names to try
    file_paths = [
        CSV_FILE_NAME,  # First preference
        JSON_FILE_NAME, # Second preference
        'mHumanEvalben_Beng.csv',  # Alternative CSV name
        'mHumanEvalben_Beng.json', # Alternative JSON name
    ]
    
    # Also check current directory for any file matching the pattern
    import glob
    for pattern in ['*HumanEval*ben*Beng*.csv', '*HumanEval*ben*Beng*.json']:
        matching_files = glob.glob(pattern)
        file_paths.extend(matching_files)
    
    # Remove duplicates
    file_paths = list(dict.fromkeys(file_paths))
    
    for file_path in file_paths:
        if os.path.exists(file_path) and os.path.isfile(file_path):
            _, ext = os.path.splitext(file_path)
            if ext.lower() in ['.csv', '.json']:
                print(f"📄 Found data file: {file_path}")
                return file_path
    
    print("❌ No suitable data file found. Please check file name and location.")
    return None

def load_data(file_path: str) -> List[Dict[str, Any]]:
    """Load data from either CSV or JSON file with robust encoding handling."""
    _, file_ext = os.path.splitext(file_path)
    file_ext = file_ext.lower()
    
    if file_ext == '.csv':
        print(f"📄 Loading CSV file: {file_path}")
        
        # Try different encodings in this order of preference
        encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
        
        for encoding in encodings_to_try:
            try:
                print(f"  Trying encoding: {encoding}")
                with open(file_path, 'r', encoding=encoding) as f:
                    reader = csv.DictReader(f)
                    data = list(reader)
                    if data:  # If we successfully read data
                        print(f"  ✅ Successfully loaded with {encoding} encoding")
                        return data
            except UnicodeDecodeError:
                print(f"  ❌ Failed with {encoding} encoding")
                continue
            except Exception as e:
                print(f"  ❌ Error reading CSV with {encoding} encoding: {str(e)}")
                continue
                
        # If all encodings fail, try binary mode with pandas if available
        try:
            print("  Trying pandas with binary mode...")
            import pandas as pd
            df = pd.read_csv(file_path)
            data = df.to_dict('records')
            print("  ✅ Successfully loaded with pandas")
            return data
        except ImportError:
            print("  ❌ Pandas not available")
        except Exception as e:
            print(f"  ❌ Error with pandas approach: {str(e)}")
            
        # If we get here, all approaches failed
        raise ValueError(f"Could not read CSV file with any attempted encoding")
        
    elif file_ext == '.json':
        print(f"📄 Loading JSON file: {file_path}")
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Handle different JSON structures
                if isinstance(data, list):
                    return data
                elif isinstance(data, dict) and 'data' in data:
                    return data['data']
                else:
                    # Try to convert to list if it's a dict with task_ids as keys
                    return [{'task_id': k, **v} for k, v in data.items()]
        except UnicodeDecodeError:
            # Try alternative encoding for JSON
            with open(file_path, 'r', encoding='latin-1') as f:
                data = json.load(f)
                # Handle different JSON structures
                if isinstance(data, list):
                    return data
                elif isinstance(data, dict) and 'data' in data:
                    return data['data']
                else:
                    # Try to convert to list if it's a dict with task_ids as keys
                    return [{'task_id': k, **v} for k, v in data.items()]
    else:
        raise ValueError(f"Unsupported file format: {file_ext}. Expected .csv or .json")

def evaluate_solutions(file_path: str, solution_column: str) -> Dict:
    """Evaluate all solutions in the given file and return results."""
    results = {}
    passed_count = 0
    total_count = 0
    failed_count = 0
    timeout_count = 0
    error_count = 0
    
    # Track progress
    start_time = time.time()
    
    print(f"🚀 Starting evaluation of {file_path}")
    print("⏳ Processing tasks...")
    
    try:
        all_rows = load_data(file_path)
        total_tasks = len(all_rows)
        
        # Extract NL and PL information from the first row
        nl_value = all_rows[0].get('nl', 'Unknown') if all_rows else 'Unknown'
        pl_value = all_rows[0].get('pl', 'Unknown') if all_rows else 'Unknown'
        
        # Setup progress tracking
        progress_interval = max(1, total_tasks // 20)  # Show progress at most 20 times
        
        for i, row in enumerate(all_rows):
            task_id = row.get('task_id', f"Task_{i}")
            prompt = row.get('prompt', '')
            solution = row.get(solution_column, '')
            test = row.get('test', '')
            
            # Skip if any of the required fields are empty
            if not prompt or not solution or not test:
                print(f"⚠️  Skipping {task_id}: Missing required fields")
                continue
            
            # Show progress indicator sparingly
            if i % progress_interval == 0 or i == total_tasks - 1:
                progress = (i + 1) / total_tasks * 100
                elapsed = time.time() - start_time
                estimated_total = elapsed / (i + 1) * total_tasks if i > 0 else 0
                remaining = max(0, estimated_total - elapsed)
                print(f"⏳ Progress: {progress:.1f}% ({i+1}/{total_tasks}) - Est. time remaining: {remaining:.1f}s", end="\r")
            
            test_file = create_test_file(task_id, prompt, solution, test)
            output, passed = run_test(test_file)
            
            # Clean up temporary file
            try:
                os.unlink(test_file)
            except:
                pass
            
            # Determine failure type
            failure_type = None
            if not passed:
                if "TIMEOUT" in output:
                    failure_type = "timeout"
                    timeout_count += 1
                elif "ERROR" in output:
                    failure_type = "error"
                    error_count += 1
                else:
                    failure_type = "fail"
                    failed_count += 1
            
            results[task_id] = {
                'passed': passed,
                'output': output,
                'failure_type': failure_type
            }
            
            total_count += 1
            if passed:
                passed_count += 1
                
        print(" " * 80, end="\r")  # Clear the progress line
    
    except Exception as e:
        print(f"\n❌ Error loading or processing file: {e}")
        return {
            'results': {},
            'total': 0,
            'passed': 0,
            'failed': 0,
            'timeout': 0,
            'error': 0,
            'pass_at_1': 0,
            'elapsed_time': time.time() - start_time,
            'nl': 'Unknown',
            'pl': 'Unknown',
            'load_error': str(e)
        }
    
    # Calculate Pass@1 score
    pass_at_1 = (passed_count / total_count) * 100 if total_count > 0 else 0
    elapsed_time = time.time() - start_time
    
    return {
        'results': results,
        'total': total_count,
        'passed': passed_count,
        'failed': failed_count,
        'timeout': timeout_count,
        'error': error_count,
        'pass_at_1': pass_at_1,
        'elapsed_time': elapsed_time,
        'nl': nl_value,
        'pl': pl_value
    }

def save_results(results, filename='evaluation_results.txt'):
    """Save the detailed results to a file."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"🔍 HUMANEVAL BENCHMARK EVALUATION RESULTS\n")
        f.write(f"📅 Evaluation completed: {timestamp}\n")
        f.write(f"⏱️  Total execution time: {results['elapsed_time']:.2f} seconds\n")
        
        # Add NL and PL information
        f.write(f"🌐 Natural Language: {results.get('nl', 'Unknown')}\n")
        f.write(f"💻 Programming Language: {results.get('pl', 'Unknown')}\n\n")
        
        # Check if there was a load error
        if 'load_error' in results and results['load_error']:
            f.write(f"❌ ERROR LOADING FILE: {results['load_error']}\n")
            return
            
        # Summary section
        f.write("📊 SUMMARY\n")
        f.write("=" * 50 + "\n")
        f.write(f"Total tasks evaluated: {results['total']}\n")
        
        if results['total'] > 0:
            f.write(f"Tasks passed:          {results['passed']} ({results['passed']/results['total']*100:.2f}%)\n")
            f.write(f"Tasks failed:          {results['failed']} ({results['failed']/results['total']*100:.2f}%)\n")
            f.write(f"Tasks timed out:       {results['timeout']} ({results['timeout']/results['total']*100:.2f}%)\n")
            f.write(f"Tasks with errors:     {results['error']} ({results['error']/results['total']*100:.2f}%)\n")
            f.write(f"✨ Pass@1 score:       {results['pass_at_1']:.2f}%\n\n")
        else:
            f.write("No tasks were successfully evaluated.\n\n")
            return
        
        # Detailed results section
        f.write("📝 DETAILED RESULTS\n")
        f.write("-" * 50 + "\n")
        
        # Group results by status
        passed_tasks = []
        failed_tasks = []
        timeout_tasks = []
        error_tasks = []
        
        for task_id, result in sorted(results['results'].items()):
            if result['passed']:
                passed_tasks.append(task_id)
            elif result['failure_type'] == 'timeout':
                timeout_tasks.append((task_id, result['output']))
            elif result['failure_type'] == 'error':
                error_tasks.append((task_id, result['output']))
            else:
                failed_tasks.append((task_id, result['output']))
        
        # Write passed tasks (just IDs)
        f.write(f"\n✅ PASSED TASKS ({len(passed_tasks)})\n")
        f.write("-" * 30 + "\n")
        for task_id in passed_tasks:
            f.write(f"{task_id}\n")
        
        # Write failed tasks with details
        f.write(f"\n❌ FAILED TASKS ({len(failed_tasks)})\n")
        f.write("-" * 30 + "\n")
        for task_id, output in failed_tasks:
            f.write(f"{task_id}: {output[:200]}...\n")
        
        # Write timeout tasks
        f.write(f"\n⏱️  TIMEOUT TASKS ({len(timeout_tasks)})\n")
        f.write("-" * 30 + "\n")
        for task_id, output in timeout_tasks:
            f.write(f"{task_id}\n")
        
        # Write error tasks
        f.write(f"\n⚠️  ERROR TASKS ({len(error_tasks)})\n")
        f.write("-" * 30 + "\n")
        for task_id, output in error_tasks:
            f.write(f"{task_id}: {output[:200]}...\n")
    
    print(f"\n📊 Detailed results saved to '{filename}'")

def print_fancy_summary(results):
    """Print a fancy summary of the evaluation results."""
    print("\n" + "=" * 60)
    print(f"✨ HUMANEVAL BENCHMARK EVALUATION SUMMARY ✨")
    print("=" * 60)
    
    # Add NL and PL information
    print(f"🌐 Natural Language (NL): {results.get('nl', 'Unknown')}")
    print(f"💻 Programming Language (PL): {results.get('pl', 'Unknown')}")
    
    # Check if there was a load error
    if 'load_error' in results and results['load_error']:
        print(f"❌ ERROR LOADING FILE: {results['load_error']}")
        print("=" * 60)
        return
        
    print(f"📊 Pass@1 Score: {results['pass_at_1']:.2f}%")
    print(f"⏱️  Execution Time: {results['elapsed_time']:.2f} seconds")
    print("-" * 60)
    
    if results['total'] > 0:
        # Calculate percentages
        pass_percent = results['passed'] / results['total'] * 100
        fail_percent = results['failed'] / results['total'] * 100
        timeout_percent = results['timeout'] / results['total'] * 100
        error_percent = results['error'] / results['total'] * 100
        
        # Create a simple bar chart with emojis
        bar_length = 30
        pass_bar = "🟩" * int(bar_length * pass_percent / 100)
        fail_bar = "🟥" * int(bar_length * fail_percent / 100)
        timeout_bar = "🟨" * int(bar_length * timeout_percent / 100)
        error_bar = "🟧" * int(bar_length * error_percent / 100)
        
        print(f"📈 Results Breakdown:")
        print(f"  ✅ Passed:   {results['passed']:3d}/{results['total']} ({pass_percent:6.2f}%) {pass_bar}")
        print(f"  ❌ Failed:   {results['failed']:3d}/{results['total']} ({fail_percent:6.2f}%) {fail_bar}")
        print(f"  ⏱️  Timeouts: {results['timeout']:3d}/{results['total']} ({timeout_percent:6.2f}%) {timeout_bar}")
        print(f"  ⚠️  Errors:   {results['error']:3d}/{results['total']} ({error_percent:6.2f}%) {error_bar}")
    else:
        print("❌ No tasks were successfully evaluated.")
    
    print("=" * 60)

# Function to run evaluation (for Jupyter notebook)
def run_evaluation(file_path=None, solution_column=SOLUTION_COLUMN):
    """
    Run the evaluation on the given file.
    
    Args:
        file_path: Path to the CSV or JSON file. If None, will try to find suitable files.
        solution_column: Column name containing the solution to evaluate.
        
    Returns:
        Dictionary with evaluation results.
    """
    # Handle the case where file_path might be a command line argument like '-f'
    if file_path is not None and (file_path.startswith('-') or not os.path.exists(file_path)):
        print(f"⚠️ Invalid file path '{file_path}', searching for data files instead...")
        file_path = None
    
    # If no file path is specified or the path is invalid, try to find a suitable file
    if file_path is None:
        file_path = find_data_file()
        
    # If we still don't have a file, return with an error
    if file_path is None:
        print("❌ No suitable data file found. Please specify a valid file path.")
        return {
            'results': {},
            'total': 0,
            'passed': 0,
            'failed': 0,
            'timeout': 0,
            'error': 0,
            'pass_at_1': 0,
            'elapsed_time': 0,
            'nl': 'Unknown',
            'pl': 'Unknown',
            'load_error': "No suitable data file found"
        }
    
    # Run the evaluation
    results = evaluate_solutions(file_path, solution_column)
    
    # Print fancy summary
    print_fancy_summary(results)
    
    # Save detailed results to a file
    save_results(results)
    
    return results

results = run_evaluation()

📄 Found data file: mHumanEval-mini.csv
🚀 Starting evaluation of mHumanEval-mini.csv
⏳ Processing tasks...
📄 Loading CSV file: mHumanEval-mini.csv
  Trying encoding: utf-8
  ❌ Failed with utf-8 encoding
  Trying encoding: latin-1
  ✅ Successfully loaded with latin-1 encoding
                                                                                
✨ HUMANEVAL BENCHMARK EVALUATION SUMMARY ✨
🌐 Natural Language (NL): ace_Arab
💻 Programming Language (PL): Unknown
📊 Pass@1 Score: 95.57%
⏱️  Execution Time: 6.03 seconds
------------------------------------------------------------
📈 Results Breakdown:
  ✅ Passed:   194/203 ( 95.57%) 🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩
  ❌ Failed:     4/203 (  1.97%) 
  ⏱️  Timeouts:   0/203 (  0.00%) 
  ⚠️  Errors:     5/203 (  2.46%) 

📊 Detailed results saved to 'evaluation_results.txt'
